# REST API

## HTTP Verbs GET & POST
[![](get_post.png)](https://youtu.be/UObINRj2EGY)

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import requests

Interacting with a REST API can be as simple as getting the data from a URL.

In this case the data is in MS Excel format.

In [ ]:
url = "https://vattenwebb.smhi.se/station/rest/report/2127"

raw_df = pd.read_excel(url,skiprows=13, names=["date","flow","quality"], na_values=['Saknas'])
df = raw_df.copy()
df = df.drop('date',axis=1)
df.index = pd.to_datetime(raw_df['date'])

In [ ]:
df.flow.plot(figsize=(18,4))

# DMI

In most cases we supply parameters in the query string and get the result in JSON format.

In [ ]:
api_key = "e11ff26d-8747-4a40-a38b-0eeec13117b9"

station_id = "9020401" # Kerteminde

# Add all parameters to URL
url = f"https://dmigw.govcloud.dk/v2/oceanObs/collections/observation/items?api-key={api_key}&stationId={station_id}&parameterId=sea_reg"
resp = requests.get(url)
resp

The requests library can handle parameters for us. Below is the same request using a dictionary instead.

In [ ]:
resp =requests.get("https://dmigw.govcloud.dk/v2/oceanObs/collections/observation/items", params={'api-key': api_key})
resp

In [ ]:
resp =requests.get("https://dmigw.govcloud.dk/v2/oceanObs/collections/observation/items", params={'api-key': api_key,
                                'stationId': station_id,
                                'parameterId': 'sea_reg'}
                                    )
resp

HTTP Status 200 is Ok

We can also check the ok property of the response object.

In [ ]:
resp.ok

There are many other HTTP status codes, but the ones in the 200 range are successful.

You can find some other status codes in the `request.codes` module.

In [ ]:
requests.codes.IM_A_TEAPOT

In [ ]:
resp.content

In [ ]:
# parse response body formatted in JSON
data = resp.json()

In [ ]:
# Select relevant fields and convert to Dataframe

ts = [{'time':p['properties']['observed'].replace("Z",""),
      'value': p['properties']['value']} for p in data['features']]
df = pd.DataFrame(ts)
df.index = pd.to_datetime(df['time'])
df = df.drop(columns=['time'])
df = df.sort_index()

In [ ]:
df.plot()

Standardized formats are easy to use, e.g. GeoJSON

In [ ]:
url = f"https://dmigw.govcloud.dk/v2/oceanObs/collections/observation/items"

resp =requests.get(url, params={'api-key': api_key,
                                'parameterId': 'sea_reg'}
                                    )
resp

In [ ]:
import folium

m =folium.Map(location=[56,10], zoom_start=7)
folium.GeoJson(resp.json()).add_to(m)
m
